In [ ]:
import os
import pandas as pd
import numpy as np
import json
import rasterio

In [ ]:
data_dir = '../data/train_test'

main = 'ref_agrifieldnet_competition_v1'

source_collection = f'{main}_source'
train_label_collection = f'{main}_labels_train'
test_label_collection = f'{main}_labels_test'

In [ ]:
with open (f'{data_dir}/{main}/{train_label_collection}/collection.json') as f:
    train_json = json.load(f)

print(train_json.keys())

dict_keys(['type', 'stac_version', 'stac_extensions', 'id', 'title', 'description', 'license', 'sci:doi', 'sci:citation', 'extent', 'links'])


In [ ]:
train_folder_ids = [i['href'].split('_')[-1].split('.')[0] for i in train_json['links'][4:]]
train_field_paths = [f'{data_dir}/{main}/{train_label_collection}/{train_label_collection}_{i}/field_ids.tif' for i in train_folder_ids]
train_label_paths = [f'{data_dir}/{main}/{train_label_collection}/{train_label_collection}_{i}/raster_labels.tif' for i in train_folder_ids]

print(train_folder_ids[0])
print(train_field_paths[0])
print(train_label_paths[0])

28852
../data/train_test/ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_28852/field_ids.tif
../data/train_test/ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_28852/raster_labels.tif


In [ ]:
competition_train_data = pd.DataFrame(train_folder_ids, columns=['unique_folder_id'])
competition_train_data['field_paths'] = train_field_paths
competition_train_data.head()

,unique_folder_id,field_paths
0,28852,../data/train_test/ref_agrifieldnet_competitio...
1,d987c,../data/train_test/ref_agrifieldnet_competitio...
2,ca1d4,../data/train_test/ref_agrifieldnet_competitio...
3,2ec18,../data/train_test/ref_agrifieldnet_competitio...
4,7575d,../data/train_test/ref_agrifieldnet_competitio...


In [ ]:
with rasterio.open(f'{data_dir}/{main}/{train_label_collection}/{train_label_collection}_{train_folder_ids[0]}/field_ids.tif') as src:
    field_data = src.read()

print(field_data.shape)
print(np.unique(field_data, return_counts=True))

field_data = field_data[0]

(1, 256, 256)
(array([   0,  756,  757, 1372, 1374], dtype=uint16), array([65461,    14,    21,    38,     2]))


In [ ]:
from collections import defaultdict

field_crops = defaultdict(lambda : [])

In [ ]:
with rasterio.open(f'{data_dir}/{main}/{train_label_collection}/{train_label_collection}_{train_folder_ids[0]}/raster_labels.tif') as src:
    crop_data = src.read()

print(crop_data.shape)
print(np.unique(crop_data, return_counts=True))

# zero because we index a one-dimension input
crop_data = crop_data[0]

(1, 256, 256)
(array([0, 1, 5, 6], dtype=uint16), array([65461,     2,    38,    35]))


In [ ]:
for x in range(crop_data.shape[0]):
    for y in range(crop_data.shape[1]):
        # get the field id and label of the pixel
        field_id = str(field_data[x][y])
        field_crop = crop_data[x][y]

        if field_crop not in field_crops[field_id]:
            field_crops[field_id].append(field_crop)

print(field_crops)

defaultdict(<function <lambda> at 0x7f47fb539900>, {'0': [0], '757': [6], '756': [6], '1372': [5], '1374': [1]})


In [ ]:
field_crop_map  =[[k, v[0]]  for k, v in field_crops.items() ]
# why first item?
print(field_crop_map)

[['0', 0], ['757', 6], ['756', 6], ['1372', 5], ['1374', 1]]


In [ ]:
field_crop = pd.DataFrame(field_crop_map , columns=['field_id','crop_id'])
field_crop.head()

,field_id,crop_id
0,0,0
1,757,6
2,756,6
3,1372,5
4,1374,1


In [ ]:
field_crop[field_crop['field_id']!='0']

,field_id,crop_id
1,757,6
2,756,6
3,1372,5
4,1374,1


### Pixel-based Random Forest model

In [ ]:
Full_bands = ['B01', 'B02', 'B03', 'B04','B05', 'B06', 'B07', 'B08','B8A', 'B09', 'B11', 'B12']

img_sh = 256
n_selected_bands= len(Full_bands[:3])
n_obs = 1  # imagery per chip (no time series)

X = np.empty((0, n_selected_bands * n_obs))
X_tile = np.empty((img_sh * img_sh, 0))
X_arrays = []

field_src = rasterio.open(competition_train_data['field_paths'].values[0])
field_array = field_src.read(1)
field_array.shape

(256, 256)

In [ ]:
field_ids = np.empty((0, 1))
print(field_ids.shape)

field_ids = np.append(field_ids, field_array.flatten())
print(field_ids.shape)
print(np.unique(field_ids, return_counts=True))

(0, 1)
(65536,)
(array([   0.,  756.,  757., 1372., 1374.]), array([65461,    14,    21,    38,     2]))


In [91]:
bands_src = [rasterio.open(f'{data_dir}/{main}/{source_collection}/{source_collection}_{competition_train_data["unique_folder_id"][0]}/{band}.tif') for band in Full_bands[:3]]

In [92]:
bands_array = [np.expand_dims(band.read(1).flatten(), axis=1) for band in bands_src]

[<open DatasetReader name='../data/train_test/ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_source/ref_agrifieldnet_competition_v1_source_28852/B01.tif' mode='r'>,
 <open DatasetReader name='../data/train_test/ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_source/ref_agrifieldnet_competition_v1_source_28852/B02.tif' mode='r'>,
 <open DatasetReader name='../data/train_test/ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_source/ref_agrifieldnet_competition_v1_source_28852/B03.tif' mode='r'>]

In [ ]:
bands_array = [np.expand_dims(band.read(1).flatten(), axis=1) for band in bands_src]